# Annotating your Data

In [ ]:
import holoviews as hv
import holoviews.util
hv.extension('bokeh')

One of the fundamental concepts in HoloViews introduced in the [Getting Started guide](../getting_started/1-Introduction.ipynb) is that of annotating data with key, semantic metadata. This user guide documents the two main types of annotation:
  1. dimensions, used to specify the abstract space in which the data resides;
  2. the element group/label system, used to organize and select data.

## Specifying dimensionality

Simple data structures such as dataframes, arrays, lists, or dictionaries cannot be given a suitable visual representation without some associated semantic context. Fundamentally, HoloViews lets you specify this context by first selecting a suitable element type (see the [Reference Gallery](http://holoviews.org/reference/index.html)), and then specifying the corresponding *dimensions*.

Here is a very simple example, showing a ``Curve`` element:

In [ ]:
xs = range(-10,11)
ys = [100-el**2 for el in xs]
curve = hv.Curve((xs, ys))
curve

All elements (except a small number of annotation elements) have two types of dimensions: *key dimensions* (``kdims``) and *value dimensions* (``vdims``). The *key dimensions* are the dimensions you can index *by* to get the values corresponding to the *value* dimensions. You can learn more about indexing data in the later [Indexing and Selecting Data](./09-Indexing_and_Selecting_Data.ipynb) user guide.

Different elements have different numbers of required key dimensions and value dimensions. For instance, a ``Curve`` always has one key dimension and one value dimension. As we did not explicitly specify anything regarding dimensions when declaring the curve above, the ``kdims`` and ``vidms`` use their default names 'x' and 'y':

In [ ]:
"Object 'curve' has kdims {kdims} and vdims {vdims} ".format(kdims=curve.kdims, vdims=curve.vdims)

The easiest way to specify dimensions other than the defaults is as strings, which sets the dimension names:

In [ ]:
trajectory = hv.Curve((xs, ys), kdims=['distance'], vdims=['height'])
trajectory

In [ ]:
"Object 'trajectory' has kdims {kdims} and vdims {vdims} ".format(kdims=trajectory.kdims, vdims=trajectory.vdims)

We can see our strings have been 'promoted' to dimension objects describing the space in which our trajectory data resides: the ``kdims`` and ``vdims`` *always* contain instances of the ``Dimension`` class, described in the following section. Note that naming our dimensions has given the corresponding visual representation appropriate axis labels.

### Dimension parameters

``Dimension`` objects have a number of parameters used to describe the space in which the data resides. Only two of these are considered *core* parameters that identify the dimension object; the rest are auxilliary metadata. Here are the descriptions of the most important ones:


<br>
<dl class="dl-horizontal">
  <dt>``name``</dt><dd>(core) A concise name for the dimension that should be useable as a keyword argument</dd>
  <dt>``label`` <dd>(core) A longer description of the dimension (can contain unicode)</dd>
  <dt>``range`` <dd>The minumum and maximum allowable values for the dimension.</dd>
  <dt>``soft_range`` <dd>Suggested minumum and maximum values, used to specify a useful portion of the range.</dd>
  <dt>``step`` <dd>If specified, the step parameter suggests an appropriate sampling of a continuous range</dd>
  <dt>``unit`` <dd>If specified, the name of the unit associated with the dimension.</dd>
  <dt>``values`` <dd>Explicit list of allowed dimension values</dd>
</dl>


For the full list of parameters, you can call ``hv.help(hv.Dimension)``.

Instead of just a string name, you can use a ``(name, label)`` tuple if you want to specify both ``name`` and ``label`` without building an explicit ``Dimension`` for ``kdims`` and ``vdims``:

In [ ]:
wo_unit = hv.Curve((xs, ys), 
                   kdims=[('distance','Horizontal distance')], 
                   vdims=[('height','Height above sea-level')])

distance = hv.Dimension('distance', label='Horizontal distance', unit='m')
height = hv.Dimension(('height','Height above sea-level'), unit='m')
with_unit = hv.Curve((xs, ys), kdims=[distance], vdims=[height])

# (using + to compose elements is described in the next guide)
wo_unit + with_unit

Note that after supplying the longer labels, you still use the short name to specify the dimension as keyword arguments. For instance,try using ``with_unit.select(distance=(5,8))`` in the cell above.

### Setting properties with redim

Declaring dimension objects with appropriate parameters can be verbose if you only want to set a few specific parameters. You can often avoid declaring explicit dimension objects using the ``redim`` method, which returns a *clone* of the element: the same data, wrapped in a new instance of the same element type with the new dimension settings.

Let's use ``redim`` to swap out the 'height' dimension for the 'altitude' dimension:

In [ ]:
renamed_height = trajectory.redim(height='altitude')
renamed_height

The ``redim`` 'method' is actually a utility that can be used to set any of the dimension parameters:

In [ ]:
renamed_height.redim.label(altitude='Altitude above sea-level', distance='Horizontal distance')

This pattern can be used to set any of the parameters listed above (unit, range, values, etc) by specifying the dimension name and the new value for the parameter.

## Organizing your elements

A complex visualization with HoloViews may include many instances of the same element where each element has a different meaning. It is useful to be able to assign semantic tags to individual elements which HoloViews does with ``group`` and ``label``:

In [ ]:
low_ys = [25-(0.5*el)**2 for el in xs]
hv.Curve((xs, low_ys), group='Trajectory', label='Shallow') + \
hv.Curve((xs, ys), group='Trajectory', label='Medium')

As shown above, ``group`` and ``label`` have been used generate sensible titles. The ``group`` is expected to specify what the element corresponds to at the level of its type, while the ``label`` is used to specify a particular element in a group.

The ``group`` and ``label`` system also allows [Customizing Plots](./03-Customizing_Plots.ipynb), and it is used for indexing, as we will see in the following [Composing_Elements](./02-Composing_Elements.ipynb) guide.